In [12]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import random 

choice = random.randint(0, 4)
ignore = ""
if choice == 0:
  ignore = 'cast'
if choice == 1:
  ignore = 'director'
if choice == 2:
  ignore = 'country'
if choice == 3:
  ignore = 'genre'

print(ignore)

# Get the data
column_names = ['show_id', 'type', 'title', 'director', 'cast', 'country', 
                'date_added', 'release_year', 'rating', 'duration', 'genres', 'description']
  
path = '/content/NetFlix.csv'
  
df = pd.read_csv(path, sep=',', names=column_names, index_col=False)

# filter data that is corresponding to shows - only want movies 
df2 = df[df['type'] != 'TV Show'].reset_index(drop=True)
data = df2.dropna().reset_index(drop=True)
data = data.tail(-1).reset_index(drop=True)
#data["key_words"] = ''
data = data.reset_index()

descriptors = ['cast', 'country', 'genres', 'director']
from ast import literal_eval # reads the string as the type of object that it represents

for cat in descriptors:
  data[cat] = data[cat].str.strip('()').str.lower().str.split(',')

sim_subsets = [['cast', 'country', 'genre','director'], 
               ['director', 'country', 'genre'], 
               ['director', 'cast', 'genre'], 
               ['director', 'cast', 'country']]
def pick_some(lst):
  if len(lst) <= 5: 
    return lst
  else:
    return lst[:5]

for cat in descriptors:
    data[cat] = data[cat].apply(pick_some)
  
def remove_space(lst):
  lst = [x.replace(' ', '') for x in lst]
  return lst

for cat in descriptors:
    data[cat] = data[cat].apply(remove_space)

#source: https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system/notebook
def create_soup(x):
    return  ' ' + ' '.join(x['cast']) + ' ' + ' '.join(x['director']) + ' ' + ' '.join(x['genres'])+ ' ' + ' '.join(x['country'])

def create_cast_soup(x):
    return  ' ' + ' '.join(x['director']) + ' ' + ' '.join(x['genres'])+ ' ' + ' '.join(x['country'])

def create_director_soup(x):
    return  ' ' + ' '.join(x['cast']) + ' ' + ' '.join(x['genres'])+ ' ' + ' '.join(x['country'])

def create_genre_soup(x):
    return  ' ' + ' '.join(x['cast']) + ' ' + ' '.join(x['director']) + ' ' + ' '.join(x['country'])

def create_country_soup(x):
    return  ' ' + ' '.join(x['cast']) + ' ' + ' '.join(x['director']) + ' ' + ' '.join(x['genres'])+ ' '

data['soup'] = data.apply(create_soup, axis=1)

new_data = data.loc[:, data.columns != ignore]

if choice == 0:
  new_data['soup'] = data.apply(create_cast_soup, axis=1)
if choice == 1:
  new_data['soup'] = data.apply(create_director_soup, axis=1)
if choice == 2:
  new_data['soup'] = data.apply(create_country_soup, axis=1)
if choice == 3:
  new_data['soup'] = data.apply(create_genre_soup, axis=1)

# Import CountVectorizer and create the count matrix
#source: https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system/notebook

from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(data['soup'])
new_count_matrix = count.fit_transform(new_data['soup'])

# Compute the Cosine Similarity matrix based on the count_matrix
#source: https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system/notebook

from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
new_cosine_sim2 = cosine_similarity(new_count_matrix, new_count_matrix)

# Reset index of our main DataFrame and construct reverse mapping as before

data = data.reset_index()
new_data = new_data.reset_index()
indices = pd.Series(data.index, index=data['title'])
new_indices = pd.Series(new_data.index, index=data['title'])

#source: https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system/notebook
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim2):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df2['title'].iloc[movie_indices]

get_recommendations('1920', cosine_sim2)
get_recommendations('1920', new_cosine_sim2)

print(get_recommendations('1920', cosine_sim2))
print(get_recommendations('1920', new_cosine_sim2))


director


<ipython-input-12-6e09eddad3b0>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['soup'] = data.apply(create_director_soup, axis=1)


110                                             Burn Out
204                                    Chalay Thay Saath
397     Crouching Tiger, Hidden Dragon: Sword of Destiny
1133                                            Good Sam
1522                                Inglourious Basterds
1687                         Joaquín Reyes: Una y no más
1874                                             Knowing
2593                                        Mystic Pizza
2659                                             Nibunan
2740                                      Ocean's Eleven
Name: title, dtype: object
110                                             Burn Out
2468            Monty Python: Live at The Hollywood Bowl
204                                    Chalay Thay Saath
397     Crouching Tiger, Hidden Dragon: Sword of Destiny
923                                For the Love of Benji
1133                                            Good Sam
1155                                    Grass Is Greener
1522

In [ ]:
'''#from gensim.summarization import keywords
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


data['nlp_data'] = data.apply(lambda row: nlp(row.description.lower().replace('\n', ' ')), axis=1)
data'''

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres,description,nlp_data
0,0,s10,Movie,1920,[vikram bhatt],"[rajneesh duggal, adah sharma, indraneil sen...",[india],15-Dec-17,2008,TV-MA,143,"[horror movies, international movies, thrill...",An architect and his wife move into a castle t...,[]
1,1,s100,Movie,3 Heroines,[iman brotoseno],"[reza rahadian, bunga citra lestari, tara ba...",[indonesia],05-Jan-19,2016,TV-PG,124,"[dramas, international movies, sports movies]",Three Indonesian women break records by becomi...,[]
2,2,s1000,Movie,Blue Mountain State: The Rise of Thadland,[lev l. spiro],"[alan ritchson, darin brooks, james cade, r...",[united states],01-Mar-16,2016,R,90,[comedies],New NFL star Thad buys his old teammates' belo...,[]
3,3,s1002,Movie,Blue Ruin,[jeremy saulnier],"[macon blair, devin ratray, amy hargreaves, ...","[united states, france]",25-Feb-19,2013,R,90,"[independent movies, thrillers]",Bad news from the past unhinges vagabond Dwigh...,[]
4,4,s1003,Movie,Blue Streak,[les mayfield],"[martin lawrence, luke wilson, peter greene,...","[germany, united states]",01-Jan-21,1999,PG-13,94,"[action & adventure, comedies]",A jewel thief returns to his hiding place afte...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4668,4668,s991,Movie,Blood Will Tell,"[miguel cohan, miguel cohan]","[oscar martínez, dolores fonzi, diego velázq...","[argentina, united states]",21-Jun-19,2019,TV-MA,113,"[dramas, independent movies, international m...",Family patriarch Elías begins to unravel after...,"[{'entity': 'B-PER', 'score': 0.5623788, 'inde..."
4669,4669,s994,Movie,Blow,[ted demme],"[johnny depp, penélope cruz, franka potente,...",[united states],01-Oct-19,2001,R,123,[dramas],Cocaine smuggler George rises from poverty to ...,"[{'entity': 'B-LOC', 'score': 0.55067974, 'ind..."
4670,4670,s997,Movie,Blue Is the Warmest Color,[abdellatif kechiche],"[léa seydoux, adèle exarchopoulos, salim kec...","[france, belgium, spain]",26-Aug-16,2013,NC-17,180,"[dramas, independent movies, international m...","Determined to fall in love, 15-year-old Adele ...",[]
4671,4671,s998,Movie,Blue Jasmine,[woody allen],"[cate blanchett, sally hawkins, alec baldwin...",[united states],08-Mar-19,2013,PG-13,98,"[comedies, dramas, independent movies]",The high life leads to high anxiety for a fash...,[]
